In [192]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier

# Metrics & Corss-Validation
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.externals import joblib
from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score,
                             fbeta_score, make_scorer, classification_report, confusion_matrix)

In [193]:
%store -r full_dataset

In [194]:
full_dataset.keys()

dict_keys(['TBL_TR_Accident_Driver', 'tbl_CR_Victim_Info', 'TBL_TR_Offence_details', 'tbl_TR_Offence', 'TBL_TR_Person_Details', 'TBL_TR_Accident_Casualties', 'tbl_CR_FIR_Details', 'Masters', 'TR_Accident_Vehicle', 'TBL_TR_Offence_Master', 'tbl_tr_accident_report', 'tbl_CR_Arr_Person_Details', 'TR_Accident_Vehicle_Master'])

In [195]:
full_dataset['TBL_TR_Accident_Driver']['TBL_TR_Accident_Driver']['Alcohol_Drugs'].value_counts()

0       434948
6515     30797
6252     22175
6253       991
6254       138
Name: Alcohol_Drugs, dtype: int64

In [197]:
accidents = pd.read_csv('accidents_report.csv')

/home/ubuntu/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (115,116) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [198]:
accidents.head()

,Unnamed: 0,Unit_ID,Crime_No,Accident_DateTime,Fatal_NonFatal,Accident_Classification,Severity,Collision_Type,Accident_Spot,Accident_Location,...,Hit_Run_val,Accident_City_val,Accident_Road_val,Lane_Type_val,Road_Markings_val,Spot_Conditions_val,Side_Walk_val,Accident_Location_id_val,RoadJunction_val,Accident_spotb_val
0,0,857,2015000096,NaN,0.0,6274,6112,6091,6083,6148,...,Yes,NaN,NaN,NaN,NaN,NaN,NaN,Residential area,NaN,NaN
1,1,613,2015000098,NaN,NaN,6274,6112,6716,6083,6150,...,Not Applicable,NaN,NaN,NaN,NaN,NaN,NaN,Open area,NaN,NaN
2,2,1898,2015000101,NaN,0.0,6274,6113,6086,6513,6148,...,No,NaN,NaN,NaN,NaN,NaN,NaN,Open area,NaN,NaN
3,3,1461,2015000102,NaN,0.0,6274,6112,6715,6723,BELAVI LIMITS,...,Not Applicable,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,1911,2015000104,NaN,0.0,6274,6112,6093,6083,6150,...,No,NaN,NaN,NaN,NaN,NaN,NaN,Near a factory industrial area,NaN,NaN


In [309]:
accidents.columns

Index(['Unit_ID', 'Crime_No', 'Accident_DateTime', 'Fatal_NonFatal',
       'Accident_Classification', 'Severity', 'Collision_Type',
       'Accident_Spot', 'Accident_Location', 'Accident_Zone',
       ...
       'Hit_Run_val', 'Accident_City_val', 'Accident_Road_val',
       'Lane_Type_val', 'Road_Markings_val', 'Spot_Conditions_val',
       'Side_Walk_val', 'Accident_Location_id_val', 'RoadJunction_val',
       'Accident_spotb_val'],
      dtype='object', length=116)

In [200]:
accidents.drop('Unnamed: 0',axis=1,inplace=True)

In [201]:
accidents.Severity.value_counts()

6112     78105
6113     60080
6111     35958
6114     18970
6503      8082
0            2
5101         1
7908         1
9787         1
11012        1
Name: Severity, dtype: int64

In [202]:
accidents_train = accidents[(accidents['Severity']==6111) |(accidents['Severity']==6112)|(accidents['Severity']==6113)|(accidents['Severity']==6114)]

In [203]:
accidents_train.columns

Index(['Unit_ID', 'Crime_No', 'Accident_DateTime', 'Fatal_NonFatal',
       'Accident_Classification', 'Severity', 'Collision_Type',
       'Accident_Spot', 'Accident_Location', 'Accident_Zone',
       ...
       'Hit_Run_val', 'Accident_City_val', 'Accident_Road_val',
       'Lane_Type_val', 'Road_Markings_val', 'Spot_Conditions_val',
       'Side_Walk_val', 'Accident_Location_id_val', 'RoadJunction_val',
       'Accident_spotb_val'],
      dtype='object', length=116)

In [204]:
accidents_train.drop(['Fatal_NonFatal',
       'Accident_Classification', 'Severity', 'Collision_Type',
       'Accident_Spot', 'Accident_Location',
       'Junction_Control', 'Road_Character',
       'Road_Type', 'Road_Classification',
       'Seperation', 'Surface_Type', 'Surface_Condition', 'Road_Condition',
       'weather', 'Location_Type', 'Main_Cause', 'Hit_Run',
       'Accident_City', 'Lane_Type', 'Road_Markings', 'Spot_Conditions','Side_Walk',
       'Accident_Location_id', 'RoadJunction',
       'Accident_spotb'],axis=1,inplace=True)

/home/ubuntu/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [205]:
accidents_train.columns

Index(['Unit_ID', 'Crime_No', 'Accident_DateTime', 'Accident_Zone',
       'NoOfVehicle_Involved', 'Road_Width', 'Shoulder_Width', 'XY_Map',
       'X_Value', 'AccidentLocation_Map', 'Collision_Diagram', 'Y_Value',
       'Route', 'KM', 'HundredM', 'Node_Map', 'Node1', 'Node2',
       'Accident_Road', 'Reporting_Officer', 'LandMark_First',
       'LandMark_Second', 'Distance_LandMark_First',
       'Distance_LandMark_Second', 'Junction_Second_Name',
       'Distance_Second_Road', 'Research_Code', 'Accident_Description',
       'Action_Special_Note', 'UserID', 'Inspection_Officer_Name',
       'Inspection_Officer_Rank', 'Inspection_Officer_Code',
       'Actual_DateOf_Occurance', 'FromDate_Of_Occurance',
       'ToDate_Of_Occurance', 'Date_Of_Filling', 'Highway_Patrol_No',
       'Aid_Post_No', 'Aid_Post_Distance', 'Trauma_Care_Centre_No',
       'Trauma_Care_Centre_Distance', 'Checked', 'Signage', 'Modified_Unit_ID',
       'Action_Flag', 'Sync_Id', 'Sync_Id_Old', 'NoOf_Animal_Injured'

# Analyzing the variables

In [206]:
accidents_train['Accident_DateTime'].value_counts()

Series([], Name: Accident_DateTime, dtype: int64)

In [207]:
accidents_train['Accident_Zone'].value_counts()

Series([], Name: Accident_Zone, dtype: int64)

In [209]:
accidents_train['XY_Map'].isnull().sum()

191141

In [210]:
accidents_train['X_Value'].isnull().sum()

191280

In [212]:
accidents_train.drop(['Accident_DateTime', 'Accident_Zone','Road_Width', 'Shoulder_Width', 'XY_Map',
       'X_Value', 'AccidentLocation_Map', 'Collision_Diagram', 'Y_Value',
       'Route', 'KM', 'HundredM', 'Node_Map', 'Node1', 'Node2'],axis=1,inplace=True)

/home/ubuntu/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [214]:
accidents_train.drop(['Reporting_Officer','LandMark_First','LandMark_Second','Distance_LandMark_First','Distance_LandMark_Second'],axis=1,inplace=True)

/home/ubuntu/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [216]:
accidents_train['Research_Code'].value_counts()

Series([], Name: Research_Code, dtype: int64)

In [217]:
accidents_train['Research_Code'].value_counts()

Series([], Name: Research_Code, dtype: int64)

In [218]:
cols_delete = ['Distance_Second_Road','Research_Code','Action_Special_Note','Inspection_Officer_Name','Inspection_Officer_Rank',
       'Inspection_Officer_Code','Actual_DateOf_Occurance','FromDate_Of_Occurance','ToDate_Of_Occurance','Date_Of_Filling',
       'Highway_Patrol_No','Aid_Post_No','Aid_Post_Distance','Trauma_Care_Centre_No','Trauma_Care_Centre_Distance', 'Checked',
       'Signage','Modified_Unit_ID','Action_Flag','Sync_Id','Sync_Id_Old']

In [219]:
accidents_train.drop(cols_delete,axis=1,inplace=True)

/home/ubuntu/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [222]:
accidents_train['FootBridgeSubway'].value_counts()

0.0    1789
Name: FootBridgeSubway, dtype: int64

In [223]:
accidents_train['ZebraCrossing'].value_counts()

0.0    1789
Name: ZebraCrossing, dtype: int64

In [224]:
accidents_train['Junction_Second_Name'].value_counts()

Series([], Name: Junction_Second_Name, dtype: int64)

In [225]:
accidents_train['Chainage'].value_counts()

1.0        6
2.0        4
60.0       4
100.0      3
12.0       2
73.0       1
0.0        1
50.0       1
3.0        1
69.0       1
70.0       1
52.0       1
4.0        1
16100.0    1
36.0       1
13.0       1
15.0       1
7.0        1
40.0       1
22.0       1
21.0       1
38.0       1
Name: Chainage, dtype: int64

In [226]:
accidents_train['PotHoles'].value_counts()

0.0    1789
Name: PotHoles, dtype: int64

In [227]:
accidents_train['Collision_Typeb'].value_counts()

6093.0     739
6085.0     231
11037.0    186
6086.0     138
6715.0      99
6525.0      96
6088.0      67
20105.0     65
11035.0     35
6090.0      24
6087.0      23
6716.0      23
6084.0      18
11036.0     15
6089.0      14
6092.0       9
6713.0       7
Name: Collision_Typeb, dtype: int64

In [228]:
accidents_train.drop(['PotHoles','Chainage','FootBridgeSubway','ZebraCrossing','Junction_Second_Name'],axis=1,inplace=True)

/home/ubuntu/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [230]:
full_dataset.keys()

dict_keys(['TBL_TR_Accident_Driver', 'tbl_CR_Victim_Info', 'TBL_TR_Offence_details', 'tbl_TR_Offence', 'TBL_TR_Person_Details', 'TBL_TR_Accident_Casualties', 'tbl_CR_FIR_Details', 'Masters', 'TR_Accident_Vehicle', 'TBL_TR_Offence_Master', 'tbl_tr_accident_report', 'tbl_CR_Arr_Person_Details', 'TR_Accident_Vehicle_Master'])

In [231]:
df_vehicle_master = full_dataset['TR_Accident_Vehicle_Master']['TR_Accident_Vehicle_Master']

In [232]:
df_vehicle = full_dataset['TR_Accident_Vehicle']['TR_Accident_Vehicle']

In [233]:
df_vehicle.head()

,Unit_ID,Registration_Number,Crime_No,Vehicle_Owner_ID,UserID,Modified_Unit_ID,Action_Flag,Sync_Id,Sync_Id_Old,DEDT
0,57,AP26AY1382,2018000071,2018000397,4400061,0,N,-1,-1,2018-07-25 18:50:56.283
1,57,AP26AY1382,2018000071,2018000397,4400061,0,N,-1,-1,2018-07-25 18:50:56.283
2,57,AP28CK8490,2017000064,2017002647,4400014,0,N,-1,-1,2017-09-25 18:17:44.780
3,57,HR38Q9825,2014000243,2014001174,4000039,0,N,-1,-1,2014-12-01 12:28:17.580
4,57,KA-09-EU-5865,2014000111,2014000332,4100020,0,N,-1,-1,2014-04-22 21:01:06.303


In [234]:
df_vehicle_master.head()

,Unit_ID,Registration_Number,Crime_No,Vehicle_Type,Vehicle_Vanoeuvre,vehicle_Make,Vehicle_Model,Vehicle_Color,Vehicle_Load,VehicleSeized,...,Vehicle_UnitID,IsGov_Vehicle,UserID,Misc_PetitionNo,Action_Taken,TrafficViolation,VehicleTypeMorth,Disposition,MechanicalFailure,IsImpacting_Vehicle
0,1761,KA-03-EZ-9285,NaN,4147,0,9032,2006,0,0,False,...,0,False,96500019,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,416,Ka28A4304,NaN,4149,0,9065,2010,0,0,False,...,0,False,7100055,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1620,KA02AF1813,NaN,4149,0,9062,2015,0,0,False,...,0,False,142500084,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,885,KL56C2187,NaN,4149,0,9065,2008,0,0,False,...,0,False,26800029,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,116,KA17V3313,NaN,4147,0,9034,2006,4171,6555,False,...,0,False,14700016,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [235]:
df_vehicle_master.columns

Index(['Unit_ID', 'Registration_Number', 'Crime_No', 'Vehicle_Type',
       'Vehicle_Vanoeuvre', 'vehicle_Make', 'Vehicle_Model', 'Vehicle_Color',
       'Vehicle_Load', 'VehicleSeized', 'Vehicle_Movement_From',
       'Vehicle_Movement_To', 'PropertyForm', 'Fuel_Type', 'HP',
       'Seating_Capacity', 'Class_Of_Vehicle', 'Unladen_Weight',
       'Registered_Laden_Weight', 'Skid_Length', 'Defect_Details',
       'Damage_Details', 'Safety_features', 'Vehicle_UnitID', 'IsGov_Vehicle',
       'UserID', 'Misc_PetitionNo', 'Action_Taken', 'TrafficViolation',
       'VehicleTypeMorth', 'Disposition', 'MechanicalFailure',
       'IsImpacting_Vehicle'],
      dtype='object')

In [236]:
df_vehicle_master.drop(['Engine_Number','Chassis_Number','IsPoliceVehicle','Modified_Unit_ID', 'Action_Flag', 'Sync_Id', 'Sync_Id_Old',
       'Vehicle_Sub_Type', 'DEDT'],axis=1,inplace=True)

KeyError: "['Engine_Number' 'Chassis_Number' 'IsPoliceVehicle' 'Modified_Unit_ID'\n 'Action_Flag' 'Sync_Id' 'Sync_Id_Old' 'Vehicle_Sub_Type' 'DEDT'] not found in axis"

In [237]:
df_vehicle_master.columns

Index(['Unit_ID', 'Registration_Number', 'Crime_No', 'Vehicle_Type',
       'Vehicle_Vanoeuvre', 'vehicle_Make', 'Vehicle_Model', 'Vehicle_Color',
       'Vehicle_Load', 'VehicleSeized', 'Vehicle_Movement_From',
       'Vehicle_Movement_To', 'PropertyForm', 'Fuel_Type', 'HP',
       'Seating_Capacity', 'Class_Of_Vehicle', 'Unladen_Weight',
       'Registered_Laden_Weight', 'Skid_Length', 'Defect_Details',
       'Damage_Details', 'Safety_features', 'Vehicle_UnitID', 'IsGov_Vehicle',
       'UserID', 'Misc_PetitionNo', 'Action_Taken', 'TrafficViolation',
       'VehicleTypeMorth', 'Disposition', 'MechanicalFailure',
       'IsImpacting_Vehicle'],
      dtype='object')

In [238]:
df_vehicle_master.shape

(444779, 33)

In [239]:
df_vehicle.columns

Index(['Unit_ID', 'Registration_Number', 'Crime_No', 'Vehicle_Owner_ID',
       'UserID', 'Modified_Unit_ID', 'Action_Flag', 'Sync_Id', 'Sync_Id_Old',
       'DEDT'],
      dtype='object')

In [240]:
clf_RF_base = RandomForestClassifier()

In [241]:
accidents_train

,Unit_ID,Crime_No,NoOfVehicle_Involved,Accident_Road,Accident_Description,UserID,NoOf_Animal_Injured,NoOf_Animal_Killed,DEDT,NoOfMinorInjured,...,Hit_Run_val,Accident_City_val,Accident_Road_val,Lane_Type_val,Road_Markings_val,Spot_Conditions_val,Side_Walk_val,Accident_Location_id_val,RoadJunction_val,Accident_spotb_val
0,857,2015000096,1,CHEERANAHALLI YATHAGADAHALLI ROAD,"ಮಂಡ್ಯ ಸಿಟಿ, ತಾವರಗೆರೆ 1ನೇ ಅಡ್ಡ ರಸ್ತೆ, ಚೀರನಹಳ್ಳಿ...",135200007.0,1,0,2017-04-27 13:18:06.373,NaN,...,Yes,NaN,NaN,NaN,NaN,NaN,NaN,Residential area,NaN,NaN
1,613,2015000098,1,ON GADAG HUBBALLI ROAD,ಸದರ ಘಟನಾ ಸ್ಥಳವು ಹುಬ್ಬಳ್ಳಿ ಗದಗ ರಸ್ತೆಯ ಮೇಲೆ ಕಾಶಪ...,126300006.0,3,0,2015-07-19 12:31:16.160,NaN,...,Not Applicable,NaN,NaN,NaN,NaN,NaN,NaN,Open area,NaN,NaN
2,1898,2015000101,2,"B.B road, Kadiganahalli service road near C.R ...",ಅಪಘಾತವು ಚಿಕ್ಕಜಾಲ ಸಂಚಾರ ಪೊಲೀಸ್ ಠಾಣಾ ಸರಹದ್ದಿನಲ್ಲ...,139900010.0,1,0,2015-09-10 10:08:45.007,NaN,...,No,NaN,NaN,NaN,NaN,NaN,NaN,Open area,NaN,NaN
3,1461,2015000102,1,ON HUKKERI TO SHIRAHATTI B.K ROAD,BELAVI LIMITS ON HUKKERI TO SHIRAHATTI B.K ROAD,7400021.0,2,0,2015-05-06 20:10:16.457,NaN,...,Not Applicable,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1911,2015000104,2,MYSORE-BANGALORE HIGHWAY ROAD,ಪೂರ್ವ-ಪಶ್ಚಿಮವಾಗಿ ಮೈಸೂರು-ಬೆಂಗಳೂರು ರಾಜ್ಯ ಹೆದ್ದಾರ...,107200031.0,2,0,2015-08-14 18:30:48.030,NaN,...,No,NaN,NaN,NaN,NaN,NaN,NaN,Near a factory industrial area,NaN,NaN
5,1332,2014000277,2,BANGALORE HOSUR NH7 ROAD,"BANGALORE HOSUR NH7 ROAD, NEAR LA CLASSIC HOTE...",140100102.0,1,0,2017-05-21 15:19:43.293,NaN,...,No,NaN,NaN,NaN,NaN,NaN,NaN,Near office complex,NaN,NaN
6,867,2014000286,1,STATE HIGWAY 19,NEAR KOTERAYANAKOIPALU VILLAGE,176900026.0,0,0,2014-11-05 11:33:23.633,NaN,...,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,1018,2014000286,1,GONIBEEDU GEATE,GONIBEEDU GATE,7000089.0,0,1,2017-05-12 13:05:55.753,NaN,...,Yes,NaN,NaN,NaN,NaN,NaN,NaN,Residential area,NaN,NaN
8,1889,2014000292,1,Near Madikehosur,ಮಹಜರ್ ರೀತ್ಯ,3100255.0,1,0,2017-05-02 13:43:14.463,NaN,...,No,NaN,NaN,NaN,NaN,NaN,NaN,Open area,NaN,NaN
9,1004,2014000295,1,NH ROAD 206,"KONEHOSUR VILLAGE,",60500102.0,0,0,2017-04-24 21:26:17.013,NaN,...,No,NaN,NaN,NaN,NaN,NaN,NaN,Near Bus stop,NaN,NaN


In [242]:
accidents_train['Accident_City_val'].value_counts()

Series([], Name: Accident_City_val, dtype: int64)

In [243]:
accident_var = accidents_train.drop(['Unit_ID', 'Crime_No','Accident_Road','Accident_Description','DEDT','NoOf_Animal_Killed',
                                    'NoOfMinorInjured','RoadNumber','SteepGradient', 'Footpath','Accident_City_val',
                                    'UserID','Road_typeb', 'Collision_Typeb','Fatal_NonFatal_val'],axis=1)

In [244]:
accident_var.columns

Index(['NoOfVehicle_Involved', 'NoOf_Animal_Injured', 'NoOfNonVehicle',
       'SpeedLimit', 'Accident_Classification_val', 'Severity_val',
       'Collision_Type_val', 'Accident_Spot_val', 'Accident_Location_val',
       'Junction_Control_val', 'Road_Character_val', 'Road_Type_val',
       'Road_Classification_val', 'Seperation_val', 'Surface_Type_val',
       'Surface_Condition_val', 'Road_Condition_val', 'weather_val',
       'Location_Type_val', 'Main_Cause_val', 'Hit_Run_val',
       'Accident_Road_val', 'Lane_Type_val', 'Road_Markings_val',
       'Spot_Conditions_val', 'Side_Walk_val', 'Accident_Location_id_val',
       'RoadJunction_val', 'Accident_spotb_val'],
      dtype='object')

In [245]:
Y = accident_var['Severity_val']

In [246]:
accident_var.drop(['NoOfVehicle_Involved', 'NoOf_Animal_Injured','NoOfNonVehicle','SpeedLimit'],axis=1,inplace=True)

In [247]:
accident_var = accident_var.fillna("Unknown")

In [248]:
accident_var.columns

Index(['Accident_Classification_val', 'Severity_val', 'Collision_Type_val',
       'Accident_Spot_val', 'Accident_Location_val', 'Junction_Control_val',
       'Road_Character_val', 'Road_Type_val', 'Road_Classification_val',
       'Seperation_val', 'Surface_Type_val', 'Surface_Condition_val',
       'Road_Condition_val', 'weather_val', 'Location_Type_val',
       'Main_Cause_val', 'Hit_Run_val', 'Accident_Road_val', 'Lane_Type_val',
       'Road_Markings_val', 'Spot_Conditions_val', 'Side_Walk_val',
       'Accident_Location_id_val', 'RoadJunction_val', 'Accident_spotb_val'],
      dtype='object')

In [249]:
accident_var.drop(['Accident_Classification_val','Severity_val','Accident_Road_val'],axis=1,inplace=True)

In [252]:
X = pd.get_dummies(accident_var, columns=['Collision_Type_val', 'Accident_Spot_val',
       'Accident_Location_val', 'Junction_Control_val', 'Road_Character_val',
       'Road_Type_val', 'Road_Classification_val', 'Seperation_val',
       'Surface_Type_val', 'Surface_Condition_val', 'Road_Condition_val',
       'weather_val', 'Location_Type_val', 'Main_Cause_val', 'Hit_Run_val', 'Lane_Type_val', 'Road_Markings_val',
       'Spot_Conditions_val', 'Side_Walk_val', 'Accident_Location_id_val',
       'RoadJunction_val', 'Accident_spotb_val'])


In [253]:
X.columns

Index(['Collision_Type_val_Brown ', 'Collision_Type_val_Driver-autorickshaw ',
       'Collision_Type_val_Drowned', 'Collision_Type_val_Head on',
       'Collision_Type_val_Hit and Run', 'Collision_Type_val_Hit animal',
       'Collision_Type_val_Hit bicyclist',
       'Collision_Type_val_Hit fixed object',
       'Collision_Type_val_Hit parked vehicle',
       'Collision_Type_val_Hit pedal cyclist',
       ...
       'RoadJunction_val_Y Junction', 'Accident_spotb_val_Bridge',
       'Accident_spotb_val_Construction Work / Material',
       'Accident_spotb_val_Culvert', 'Accident_spotb_val_Curve',
       'Accident_spotb_val_Other', 'Accident_spotb_val_Pot holed',
       'Accident_spotb_val_Steep Incline or Climb',
       'Accident_spotb_val_Straight and flat', 'Accident_spotb_val_Unknown'],
      dtype='object', length=229)

In [254]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=0)

In [255]:
clf_RF_base.fit(X_train, y_train)

/home/ubuntu/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [256]:
pred = clf_RF_base.predict(X_test)

In [257]:
pred

array(['Simple Injury', 'Grievous Injury', 'Grievous Injury', ...,
       'Fatal', 'Grievous Injury', 'Grievous Injury'], dtype=object)

In [259]:
print("Accuracy:",accuracy_score(y_test, pred))

Accuracy: 0.4334981746627657


In [191]:
# Print the name and gini importance of each feature
for feature in zip(X.columns, clf_RF_base.feature_importances_):
    print(feature)

('Collision_Type_val_Brown ', 1.0992483132393775e-06)
('Collision_Type_val_Driver-autorickshaw ', 0.0)
('Collision_Type_val_Drowned', 0.0036392973041335943)
('Collision_Type_val_Head on', 0.014275122572860472)
('Collision_Type_val_Hit and Run', 0.007138542325905118)
('Collision_Type_val_Hit animal', 0.001625724160723059)
('Collision_Type_val_Hit bicyclist', 0.005545247901612633)
('Collision_Type_val_Hit fixed object', 0.0035555111685704145)
('Collision_Type_val_Hit parked vehicle', 0.0027425895271407)
('Collision_Type_val_Hit pedal cyclist', 0.0010615433746233988)
('Collision_Type_val_Hit pedestrian', 0.010107143408015402)
('Collision_Type_val_Hit puchcart', 0.001225898544138192)
('Collision_Type_val_Medium', 4.483376095603848e-06)
('Collision_Type_val_Not Applicable', 0.010612072973524775)
('Collision_Type_val_Others', 0.023127846408225718)
('Collision_Type_val_Overturning', 0.004206479325034373)
('Collision_Type_val_Rear end', 0.006974663382952761)
('Collision_Type_val_Right Turn Col

In [263]:
estimator = clf_RF_base.estimators_[5]


In [265]:
accident_var.head()

,Collision_Type_val,Accident_Spot_val,Accident_Location_val,Junction_Control_val,Road_Character_val,Road_Type_val,Road_Classification_val,Seperation_val,Surface_Type_val,Surface_Condition_val,...,Location_Type_val,Main_Cause_val,Hit_Run_val,Lane_Type_val,Road_Markings_val,Spot_Conditions_val,Side_Walk_val,Accident_Location_id_val,RoadJunction_val,Accident_spotb_val
0,Hit pedestrian,Other,City/Town,Not at Junction,Straight and flat,Minor District Road,Unknown,Unknown,Bitumen(Tar),Not Applicable,...,City/Town,Human Error,Yes,Unknown,Unknown,Unknown,Unknown,Residential area,Unknown,Unknown
1,Drowned,Other,Rural Areas,Not Applicable,Not Applicable,NH,Unknown,Unknown,Bitumen(Tar),Dry,...,Rural Areas,Human Error,Not Applicable,Unknown,Unknown,Unknown,Unknown,Open area,Unknown,Unknown
2,Rear end,Not Applicable,City/Town,Not Applicable,Not Applicable,NH,Unknown,Unknown,Bitumen(Tar),Dry,...,City/Town,Human Error,No,Unknown,Unknown,Unknown,Unknown,Open area,Unknown,Unknown
3,Skidding or Self accident,Curves,Unknown,Not Applicable,Curve,Village Road,Unknown,Unknown,Bitumen(Tar),Dry,...,Rural Areas,Human Error,Not Applicable,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown
4,Others,Other,Rural Areas,Not Applicable,Incline,State Highway,Unknown,Unknown,Bitumen(Tar),Not Applicable,...,Rural Areas,Human Error,No,Unknown,Unknown,Unknown,Unknown,Near a factory industrial area,Unknown,Unknown


In [266]:
from sklearn.feature_selection import SelectFromModel


In [271]:
sfm = SelectFromModel(clf_RF_base, threshold=0.01)

# Train the selector
sfm.fit(X_train, y_train)

SelectFromModel(estimator=RandomForestClassifier(bootstrap=True,
                                                 class_weight=None,
                                                 criterion='gini',
                                                 max_depth=None,
                                                 max_features='auto',
                                                 max_leaf_nodes=None,
                                                 min_impurity_decrease=0.0,
                                                 min_impurity_split=None,
                                                 min_samples_leaf=1,
                                                 min_samples_split=2,
                                                 min_weight_fraction_leaf=0.0,
                                                 n_estimators=10, n_jobs=None,
                                                 oob_score=False,
                                                 random_state=None, verbose=0,
   

In [272]:
for feature_list_index in sfm.get_support(indices=True):
    print(X.columns[feature_list_index])

Collision_Type_val_Head on
Collision_Type_val_Hit pedestrian
Collision_Type_val_Not Applicable
Collision_Type_val_Others
Accident_Spot_val_Cross roads
Accident_Spot_val_Narrow road
Accident_Spot_val_Not Applicable
Accident_Spot_val_Other
Junction_Control_val_Controlled
Junction_Control_val_Not Applicable
Junction_Control_val_Not at  Junction
Junction_Control_val_Uncontrolled
Road_Character_val_Curve
Road_Character_val_Not Applicable
Road_Character_val_Others
Road_Character_val_Straight and flat
Road_Type_val_NH
Road_Type_val_Others
Road_Type_val_State Highway
Surface_Type_val_Bitumen(Tar)
Surface_Condition_val_Dry
Surface_Condition_val_Not Applicable
Surface_Condition_val_Others
Road_Condition_val_No influence on accident
Road_Condition_val_Not Applicable
weather_val_Clear
weather_val_Fine
weather_val_Others
Main_Cause_val_Human Error
Main_Cause_val_Not Applicable
Hit_Run_val_No
Hit_Run_val_Not Applicable
Hit_Run_val_Yes
Accident_Location_id_val_Near Bus stop
Accident_Location_id_val_O

In [270]:
sfm.get_support(indices=True)

array([], dtype=int64)

In [278]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=0)

In [279]:
clf_RF_base.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [292]:
cor = X.corr()

In [307]:
cor['Collision_Type_val_Driver-autorickshaw ']

Collision_Type_val_Brown                                  -0.000005
Collision_Type_val_Driver-autorickshaw                     1.000000
Collision_Type_val_Drowned                                -0.000278
Collision_Type_val_Head on                                -0.000784
Collision_Type_val_Hit and Run                            -0.000646
Collision_Type_val_Hit animal                             -0.000155
Collision_Type_val_Hit bicyclist                          -0.000384
Collision_Type_val_Hit fixed object                       -0.000241
Collision_Type_val_Hit parked vehicle                     -0.000194
Collision_Type_val_Hit pedal cyclist                      -0.000126
Collision_Type_val_Hit pedestrian                         -0.000762
Collision_Type_val_Hit puchcart                           -0.000129
Collision_Type_val_Medium                                 -0.000005
Collision_Type_val_Not Applicable                         -0.000710
Collision_Type_val_Others                       

In [285]:
df_vehicle_master.columns

Index(['Unit_ID', 'Registration_Number', 'Crime_No', 'Vehicle_Type',
       'Vehicle_Vanoeuvre', 'vehicle_Make', 'Vehicle_Model', 'Vehicle_Color',
       'Vehicle_Load', 'VehicleSeized', 'Vehicle_Movement_From',
       'Vehicle_Movement_To', 'PropertyForm', 'Fuel_Type', 'HP',
       'Seating_Capacity', 'Class_Of_Vehicle', 'Unladen_Weight',
       'Registered_Laden_Weight', 'Skid_Length', 'Defect_Details',
       'Damage_Details', 'Safety_features', 'Vehicle_UnitID', 'IsGov_Vehicle',
       'UserID', 'Misc_PetitionNo', 'Action_Taken', 'TrafficViolation',
       'VehicleTypeMorth', 'Disposition', 'MechanicalFailure',
       'IsImpacting_Vehicle'],
      dtype='object')

In [287]:
df_vehicle_analysis = df_vehicle_master.drop(['Unit_ID', 'Registration_Number', 'Crime_No','UserID'],axis=1)

In [288]:
accidents_train.columns

Index(['Unit_ID', 'Crime_No', 'NoOfVehicle_Involved', 'Accident_Road',
       'Accident_Description', 'UserID', 'NoOf_Animal_Injured',
       'NoOf_Animal_Killed', 'DEDT', 'NoOfMinorInjured', 'NoOfNonVehicle',
       'RoadNumber', 'SpeedLimit', 'Road_typeb', 'SteepGradient', 'Footpath',
       'Collision_Typeb', 'Fatal_NonFatal_val', 'Accident_Classification_val',
       'Severity_val', 'Collision_Type_val', 'Accident_Spot_val',
       'Accident_Location_val', 'Junction_Control_val', 'Road_Character_val',
       'Road_Type_val', 'Road_Classification_val', 'Seperation_val',
       'Surface_Type_val', 'Surface_Condition_val', 'Road_Condition_val',
       'weather_val', 'Location_Type_val', 'Main_Cause_val', 'Hit_Run_val',
       'Accident_City_val', 'Accident_Road_val', 'Lane_Type_val',
       'Road_Markings_val', 'Spot_Conditions_val', 'Side_Walk_val',
       'Accident_Location_id_val', 'RoadJunction_val', 'Accident_spotb_val'],
      dtype='object')

In [330]:
accidents_train.to_csv('accident_cleaned.csv')

In [308]:
X.to_csv('accident_categorical.csv')

In [612]:
accident_vehicle = pd.read_csv('csv_file/accident_report_vehicle_merge.csv')

In [613]:
accident_vehicle.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'Unit_ID', 'Crime_No',
       'NoOfVehicle_Involved', 'Accident_Road', 'Accident_Description',
       'UserID', 'NoOf_Animal_Injured', 'NoOf_Animal_Killed', 'DEDT',
       'NoOfMinorInjured', 'NoOfNonVehicle', 'RoadNumber', 'SpeedLimit',
       'Road_typeb', 'SteepGradient', 'Footpath', 'Collision_Typeb',
       'Fatal_NonFatal_val', 'Accident_Classification_val', 'Severity_val',
       'Collision_Type_val', 'Accident_Spot_val', 'Accident_Location_val',
       'Junction_Control_val', 'Road_Character_val', 'Road_Type_val',
       'Road_Classification_val', 'Seperation_val', 'Surface_Type_val',
       'Surface_Condition_val', 'Road_Condition_val', 'weather_val',
       'Location_Type_val', 'Main_Cause_val', 'Hit_Run_val',
       'Accident_City_val', 'Accident_Road_val', 'Lane_Type_val',
       'Road_Markings_val', 'Spot_Conditions_val', 'Side_Walk_val',
       'Accident_Location_id_val', 'RoadJunction_val', 'Accident_spotb_val',
       'uc', 'Vehicle_

In [614]:
accident_vehicle.drop(['Unnamed: 0', 'Unnamed: 0.1', 'Unit_ID', 'Crime_No',
       'NoOfVehicle_Involved', 'Accident_Road','Accident_Description',
       'UserID', 'NoOf_Animal_Injured', 'NoOf_Animal_Killed', 'DEDT',
       'NoOfMinorInjured', 'NoOfNonVehicle', 'RoadNumber', 'SpeedLimit',
       'Road_typeb', 'SteepGradient', 'Footpath', 'Collision_Typeb',
       'Fatal_NonFatal_val', 'Accident_Classification_val','Accident_City_val','Accident_Road_val'],axis=1,inplace=True)

In [616]:
accident_vehicle.columns

Index(['Severity_val', 'Collision_Type_val', 'Accident_Spot_val',
       'Accident_Location_val', 'Junction_Control_val', 'Road_Character_val',
       'Road_Type_val', 'Road_Classification_val', 'Seperation_val',
       'Surface_Type_val', 'Surface_Condition_val', 'Road_Condition_val',
       'weather_val', 'Location_Type_val', 'Main_Cause_val', 'Hit_Run_val',
       'Lane_Type_val', 'Road_Markings_val', 'Spot_Conditions_val',
       'Side_Walk_val', 'Accident_Location_id_val', 'RoadJunction_val',
       'Accident_spotb_val', 'uc', 'Vehicle_Type', 'Vehicle_Vanoeuvre',
       'Vehicle_Model', 'Vehicle_Color', 'Vehicle_Load', 'VehicleSeized',
       'Fuel_Type', 'Seating_Capacity', 'Class_Of_Vehicle', 'IsPoliceVehicle',
       'IsGov_Vehicle', 'Vehicle_Sub_Type'],
      dtype='object')

In [617]:
accident_vehicle.drop(['IsPoliceVehicle','IsGov_Vehicle','Location_Type_val','Side_Walk_val','Vehicle_Model'],axis=1,inplace=True)

In [618]:
accident_vehicle = accident_vehicle.fillna("Unknown")

In [619]:
accident_vehicle.columns

Index(['Severity_val', 'Collision_Type_val', 'Accident_Spot_val',
       'Accident_Location_val', 'Junction_Control_val', 'Road_Character_val',
       'Road_Type_val', 'Road_Classification_val', 'Seperation_val',
       'Surface_Type_val', 'Surface_Condition_val', 'Road_Condition_val',
       'weather_val', 'Main_Cause_val', 'Hit_Run_val', 'Lane_Type_val',
       'Road_Markings_val', 'Spot_Conditions_val', 'Accident_Location_id_val',
       'RoadJunction_val', 'Accident_spotb_val', 'uc', 'Vehicle_Type',
       'Vehicle_Vanoeuvre', 'Vehicle_Color', 'Vehicle_Load', 'VehicleSeized',
       'Fuel_Type', 'Seating_Capacity', 'Class_Of_Vehicle',
       'Vehicle_Sub_Type'],
      dtype='object')

In [620]:
accident_vehicle = pd.get_dummies(accident_vehicle, columns=['Collision_Type_val', 'Accident_Spot_val',
      'Accident_Location_val', 'Junction_Control_val', 'Road_Character_val',
      'Road_Type_val', 'Road_Classification_val', 'Seperation_val',
      'Surface_Type_val', 'Surface_Condition_val', 'Road_Condition_val',
      'weather_val', 'Main_Cause_val', 'Hit_Run_val', 'Lane_Type_val', 'Road_Markings_val',
      'Spot_Conditions_val', 'Accident_Location_id_val',
      'RoadJunction_val', 'Accident_spotb_val'])

In [621]:
Y_new = accident_vehicle['Severity_val']

In [622]:
X_new = accident_vehicle.drop(['Severity_val'],axis=1)

In [623]:
X_new['VehicleSeized'].value_counts()

False    133958
True      18827
Name: VehicleSeized, dtype: int64

In [624]:
X_new=X_new.drop(['Class_Of_Vehicle','Vehicle_Color','Seating_Capacity','VehicleSeized'],axis=1)

In [634]:
X_new.columns

Index(['Collision_Type_val_Driver-autorickshaw ', 'Collision_Type_val_Drowned',
       'Collision_Type_val_Head on', 'Collision_Type_val_Hit and Run',
       'Collision_Type_val_Hit animal', 'Collision_Type_val_Hit bicyclist',
       'Collision_Type_val_Hit fixed object',
       'Collision_Type_val_Hit parked vehicle',
       'Collision_Type_val_Hit pedal cyclist',
       'Collision_Type_val_Hit pedestrian',
       ...
       'Vehicle_Sub_Type_ROAD ROLLER', 'Vehicle_Sub_Type_SCOOTER',
       'Vehicle_Sub_Type_STATION WAGON', 'Vehicle_Sub_Type_TANKER',
       'Vehicle_Sub_Type_TEMPO', 'Vehicle_Sub_Type_TRACTOR',
       'Vehicle_Sub_Type_TRAIN', 'Vehicle_Sub_Type_Unknown',
       'Vehicle_Sub_Type_VAN', 'Vehicle_Sub_Type_accussed/summons'],
      dtype='object', length=302)

In [662]:
def get_accuracy(pred,y_test):
    return accuracy_score(y_test, pred)

In [625]:
X_new = pd.get_dummies(X_new, columns=['Vehicle_Type', 'Vehicle_Vanoeuvre', 'Vehicle_Load', 'Fuel_Type','Vehicle_Sub_Type'])

In [626]:
X_new.columns

Index(['uc', 'Collision_Type_val_Driver-autorickshaw ',
       'Collision_Type_val_Drowned', 'Collision_Type_val_Head on',
       'Collision_Type_val_Hit and Run', 'Collision_Type_val_Hit animal',
       'Collision_Type_val_Hit bicyclist',
       'Collision_Type_val_Hit fixed object',
       'Collision_Type_val_Hit parked vehicle',
       'Collision_Type_val_Hit pedal cyclist',
       ...
       'Vehicle_Sub_Type_ROAD ROLLER', 'Vehicle_Sub_Type_SCOOTER',
       'Vehicle_Sub_Type_STATION WAGON', 'Vehicle_Sub_Type_TANKER',
       'Vehicle_Sub_Type_TEMPO', 'Vehicle_Sub_Type_TRACTOR',
       'Vehicle_Sub_Type_TRAIN', 'Vehicle_Sub_Type_Unknown',
       'Vehicle_Sub_Type_VAN', 'Vehicle_Sub_Type_accussed/summons'],
      dtype='object', length=303)

In [627]:
X_new = X_new.drop(['uc'],axis=1)

In [628]:
X_train, X_test, y_train, y_test = train_test_split(X_new, Y_new, test_size=0.2, random_state=0)

In [629]:
clf_RF_base.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [630]:
pred = clf_RF_base.predict(X_test)

In [661]:
print("Accuracy:",get_accuracy(pred,y_test))

Accuracy: 0.5377442157279838


In [632]:
import pickle

In [633]:
filename = 'Severity_Classification/severity_model.sav'
pickle.dump(clf_RF_base, open(filename, 'wb'))

In [663]:
X_new.to_csv("final.csv")

In [664]:
Y_new.to_csv("severity.csv")

/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.


In [665]:
X_new.shape

(152785, 302)

In [666]:
Y_new.shape

(152785,)